In [ ]:
!pip install git+https://github.com/entelecheia/eKoNLPy.git
!pip install eKoNLPy

In [ ]:
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from nltk.util import everygrams
from ekonlpy.sentiment import MPCK
from ekonlpy.sentiment import MPKO
import re
from tqdm.notebook import tqdm

In [ ]:
df_tmp = pd.read_csv('/content/drive/MyDrive/BOK_금리예측_비공개/data/minutes_df.csv')
df_tmp

,dates,docs
0,2022-10-27,1. 일 자 2022년 10월 27일(목)2. 장 소 금융통화위원회 회의실3. 출석...
1,2022-10-12,1. 일 자 2022년 10월 12일(수)2. 장 소 금융통화위원회 회의실3. 출석...
2,2022-09-22,1. 일 자 2022년 9월 22일(목)2. 장 소 금융통화위원회 회의실3. 출석위...
3,2022-09-08,1. 일 자 2022년 9월 8일(목)2. 장 소 금융통화위원회 회의실3. 출석위원...
4,2022-08-25,1. 일 자 2022년 8월 25일(목)2. 장 소 금융통화위원회 회의실3. 출석위...
...,...,...
295,2008-05-08,"동향 국제국장이 보고 제 호 외환국제금융 동향 그리고 금융시장국장이, 52 - ,보..."
296,2008-04-10,본회의에서는 의장이 한국은행법 제 조에 의거하여 의안 제 호 통( ) 28 21 -...
297,2008-03-20,년도 연차보고서 안 은 공개시장조작 및 고액권 발행 준비 등 주2007 ( )요 업...
298,2008-03-07,전일 개최된 동향보고회의에서 조사국장이 보고 제 호 국내외 경제( ) 26 -동향 ...


In [ ]:
pd.concat([pd.DataFrame([], columns=['dates', 'sentences']), pd.DataFrame([[1, 2]], columns=['dates', 'sentences'])])

,dates,sentences
0,1,2


In [ ]:
df_result = pd.DataFrame([], columns=['dates', 'sentences'])
for idx in tqdm(df_tmp.index):
        date = df_tmp.loc[idx, 'dates']
        content = df_tmp.loc[idx, 'docs']
        content = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣,. ]', '', content)
        content = re.sub('\d+.\d*', '',content)
        content = re.sub("([a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", '', content)
        content = re.sub("\.+\.", '.', content)
        content = re.sub(' +', ' ', content)
        cnt_temp = []
        cnt_list = content.split('.')
        for cnt in cnt_list:
            if (cnt not in ['', ' ']) and (len(cnt) >  10) :
                df_temp = pd.DataFrame([[date, cnt]], columns=['dates', 'sentences'])
                df_result = pd.concat([df_result, df_temp])
df_result

  0%|          | 0/300 [00:00<?, ?it/s]

,dates,sentences
0,2022-10-27,일 자 목 장 소 금융통화위원회 회의실 출석위원 이 창 용 의 장총재조 윤 제 위...
0,2022-10-27,의결안건의안 제 한국은행의 금융기관대출규정 개정안 담당 부총재보가한국은행법제제 및...
0,2022-10-27,위원 토의내용제안설명에 이어 관련부서에서는 위원협의회에서 논의된 주요내용을 다음과...
0,2022-10-27,여러 위원들은 대출 적격담보증권의 범위 확대와 관련하여 동 조치가 가지는개별적 의미...
0,2022-10-27,이에 대해 관련부서에서는 최근 시장 유동성 사정이 다소 위축된 근저에 담보 이슈가 ...
...,...,...
0,2008-02-13,"우 유동성 확보 차익실현 등을 위해 현물 순매도를 큰 폭으로 늘린 데다 선물도 순,..."
0,2008-02-13,소기업대출을 중심으로 민간신용의 증가세가 크게 확대된 데다 정부부문에서도 공공자금관...
0,2008-02-13,CD월 하순 들어서는 발행이 크게 둔화되었음한편 수시입출식예금 요구불예금 포함 은...
0,2008-02-13,소기업대출은 전년말 부채비율 관리를 위해서 일시 상환되었던 대출의 재차입 수요에 부...


In [ ]:
mpck = MPCK()
df_result = pd.DataFrame([], columns=['dates', 'sentences', 'ngram_sentence'])
for idx in tqdm(df_tmp.index):
        date = df_tmp.loc[idx, 'dates']
        content = df_tmp.loc[idx, 'docs']
        content = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣,. ]', '', content)
        content = re.sub('\d+.\d*', '',content)
        content = re.sub("([a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)", '', content)
        content = re.sub("\.+\.", '.', content)
        content = re.sub(' +', ' ', content)
        cnt_temp = []
        cnt_list = content.split('.')
        for cnt in cnt_list:
            if (cnt not in ['', ' ']) and (len(cnt) >  10) :
                tokens = mpck.tokenize(cnt)
                ngrams = mpck.ngramize(tokens)
                df_temp = pd.DataFrame([[date, cnt, ngrams]], columns=['dates', 'sentences', 'ngram_sentence'])
                df_result = pd.concat([df_result, df_temp])
df_result

  0%|          | 0/300 [00:00<?, ?it/s]

,dates,sentences,ngram_sentence
0,2022-10-27,일 자 목 장 소 금융통화위원회 회의실 출석위원 이 창 용 의 장총재조 윤 제 위...,[]
0,2022-10-27,의결안건의안 제 한국은행의 금융기관대출규정 개정안 담당 부총재보가한국은행법제제 및...,[]
0,2022-10-27,위원 토의내용제안설명에 이어 관련부서에서는 위원협의회에서 논의된 주요내용을 다음과...,[]
0,2022-10-27,여러 위원들은 대출 적격담보증권의 범위 확대와 관련하여 동 조치가 가지는개별적 의미...,[]
0,2022-10-27,이에 대해 관련부서에서는 최근 시장 유동성 사정이 다소 위축된 근저에 담보 이슈가 ...,[]
...,...,...,...
0,2008-02-13,"우 유동성 확보 차익실현 등을 위해 현물 순매도를 큰 폭으로 늘린 데다 선물도 순,...",[유동성/NNG;확보/NNG]
0,2008-02-13,소기업대출을 중심으로 민간신용의 증가세가 크게 확대된 데다 정부부문에서도 공공자금관...,"[민간/NNG;신용/NNG;증가/NNG, 금리/NNG;매력/NNG;높/VA]"
0,2008-02-13,CD월 하순 들어서는 발행이 크게 둔화되었음한편 수시입출식예금 요구불예금 포함 은...,[]
0,2008-02-13,소기업대출은 전년말 부채비율 관리를 위해서 일시 상환되었던 대출의 재차입 수요에 부...,[부도율/NNG;하락/NNG]


In [ ]:
drp_idx = []
df_result = df_result.reset_index(drop=True)
for idx in df_result.index :
    if df_result.loc[idx, 'ngram_sentence'] == []:
        drp_idx.append(idx)
df_result = df_result.drop(drp_idx)
df_result

,dates,sentences,ngram_sentence
7,2022-10-27,이에 대해 관련부서에서는 현 상황에서 금리인상 기조를 유지하는 것에는 변함이 없어야...,"[유동성/NNG;위축/NNG, 한시적/VAX;완화/NNG, 금리/NNG;인상/NNG]"
11,2022-10-27,일부 위원은 회사채 스프레드spread가 상승하면서 크레딧credit 이슈가 나타날...,[회사채/NNG;스프레드/NNG;상승/NNG]
12,2022-10-27,또한 현재로서는 은행의 건전성 수준 등을 고려할 때 시장기능의 일부 위축에도 불구...,[고유동성/NNG;자산/NNG;확보/NNG]
13,2022-10-27,일부 위원은 현재 고물가에 대응하여 금리를 올리는 상황에서 경기와 물가간상충관계가 ...,[금리/NNG;오르/VV]
22,2022-10-27,"일부 위원은 현 경제상황에서 한국은행의 유동성 공급 필요성은 충분해 보이나, 긴축적...",[ecb/NNG;금리/NNG;인상/NNG]
...,...,...,...
48130,2008-02-13,신용등급이 높은 기업 위주로 대출이 취급되면서 상승폭이 일 금리 상승폭을CD상당폭 ...,"[악화/NNG;경기/NNG;침체/NNG;우려/NNG, 주택담보대출/NNG;금리/NN..."
48131,2008-02-13,"우 유동성 확보 차익실현 등을 위해 현물 순매도를 큰 폭으로 늘린 데다 선물도 순,...",[유동성/NNG;확보/NNG]
48132,2008-02-13,소기업대출을 중심으로 민간신용의 증가세가 크게 확대된 데다 정부부문에서도 공공자금관...,"[민간/NNG;신용/NNG;증가/NNG, 금리/NNG;매력/NNG;높/VA]"
48134,2008-02-13,소기업대출은 전년말 부채비율 관리를 위해서 일시 상환되었던 대출의 재차입 수요에 부...,[부도율/NNG;하락/NNG]


In [ ]:
df_result = df_result.reset_index(drop=True)
df_result.to_csv('/content/drive/MyDrive/BOK_금리예측/minutes_dp_sentence_split.csv', index=False)